In [1]:
import cv2
import numpy as np
import os
import mediapipe as mp

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from scipy import stats
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

from jproperties import Properties

In [2]:
configs = Properties()
with open('../.properties', 'rb') as config_file: configs.load(config_file)
CAMERA_INDEX = int(configs.get('CAMERA_INDEX').data)

In [3]:
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
DATA_PATH = os.path.join('dist') 
num_videos = 20
sequence_length = 60

if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)

In [5]:
# clear dist
for file in os.listdir(DATA_PATH):
    os.remove(os.path.join(DATA_PATH, file))

# Registraiamo video

In [24]:
cap = cv2.VideoCapture(CAMERA_INDEX)

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

width= int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

for idx_video in range(num_videos):

    video_filename = os.path.join(DATA_PATH, f"{idx_video}.mp4")
    writer= cv2.VideoWriter(video_filename, cv2.VideoWriter_fourcc(*'DIVX'), 20, (width,height))

    for idx_frame in range(sequence_length):
        ret, frame = cap.read()

        if idx_frame == 0:
            cv2.putText(frame, f'Stating Video #{idx_video+1}', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
            cv2.imshow('OpenCV Feed', frame)
            cv2.waitKey(3000)
        else:
            cv2.imshow('OpenCV Feed', frame)

        writer.write(frame)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    writer.release()
                
cap.release()
cv2.destroyAllWindows()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Open

## Mostriamo e basta

In [ ]:
cap = cv2.VideoCapture(CAMERA_INDEX)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Converti il frame in scala di grigi
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Esegui il rilevamento delle pose con MediaPipe
    results = holistic.process(frame_rgb)

    # Estrai le coordinate della spalla destra, del gomito destro e della mano destra
    if results.pose_landmarks:
        right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
        right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]

        # Le coordinate sono normalizzate, quindi convertile in coordinate pixel
        height, width, _ = frame.shape
        shoulder_x, shoulder_y = int(right_shoulder.x * width), int(right_shoulder.y * height)
        elbow_x, elbow_y = int(right_elbow.x * width), int(right_elbow.y * height)
        wrist_x, wrist_y = int(right_wrist.x * width), int(right_wrist.y * height)

        # Disegna i punti sul frame
        cv2.circle(frame, (shoulder_x, shoulder_y), 5, (0, 0, 255), -1)
        cv2.circle(frame, (elbow_x, elbow_y), 5, (0, 255, 0), -1)
        cv2.circle(frame, (wrist_x, wrist_y), 5, (255, 0, 0), -1)

    # Visualizza il frame con le coordinate
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Modello

In [5]:
window_len = 10

In [10]:
X_training = []
y_training = []
frame_buffer = []
video_paths = [os.path.join(DATA_PATH, f"{idx_video}.mp4") for idx_video in range(num_videos)]

for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

        if results.pose_landmarks:
            right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
            right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]
            
            # Estrai le coordinate x, y, e z e normalizzale
            shoulder_coords = [
                right_shoulder.x, 
                right_shoulder.y, 
                #right_shoulder.z
            ]
            elbow_coords = [
                right_elbow.x, 
                right_elbow.y, 
                #right_elbow.z
            ]
            wrist_coords = [
                right_wrist.x, 
                right_wrist.y, 
                #right_wrist.z
            ]
            print(wrist_coords)

            frame_buffer.append(
                shoulder_coords
                + elbow_coords 
                + wrist_coords
            )

            # Se il buffer contiene abbastanza frame, aggiungilo a X_training e y_training
            if len(frame_buffer) >= window_len:
                X_training.append(frame_buffer[:int(window_len/2)])
                y_training.append(frame_buffer[-1])
                frame_buffer = frame_buffer[1:]

        cv2.imshow('OpenCV Feed', frame)

    cap.release()


[0.5820856690406799, 1.7001038789749146]
[0.5756016969680786, 1.699339747428894]
[0.5722960829734802, 1.6982308626174927]
[0.5726333260536194, 1.696420431137085]
[0.5729967951774597, 1.6958287954330444]
[0.5728054642677307, 1.6960071325302124]
[0.570515513420105, 1.6950948238372803]
[0.5682741403579712, 1.6914844512939453]
[0.5685549974441528, 1.6894422769546509]
[0.5683521032333374, 1.6766304969787598]
[0.5608051419258118, 1.6786268949508667]
[0.5606741309165955, 1.681058406829834]
[0.5558356642723083, 1.6902120113372803]
[0.5525819063186646, 1.6928410530090332]
[0.5529975891113281, 1.6905521154403687]
[0.552775502204895, 1.6917952299118042]
[0.5528267025947571, 1.6915920972824097]
[0.5531741380691528, 1.6915584802627563]
[0.5533888936042786, 1.6908857822418213]
[0.5377687215805054, 1.691857099533081]
[0.5049334764480591, 1.6644526720046997]
[0.2759847342967987, 0.9955483078956604]
[0.2725866138935089, 0.9187784790992737]
[0.28252944350242615, 0.8848096132278442]
[0.29670727252960205,

In [11]:
X_training = np.array(X_training)
y_training = np.array(y_training)
X_training.shape, y_training.shape

((1190, 5, 6), (1190, 6))

In [14]:
# Define the model
model = Sequential()

# Add an LSTM layer with input shape (.., ..)
model.add(LSTM(64, input_shape=(X_training.shape[1], X_training.shape[2]), activation='relu', return_sequences=True))

# Add another LSTM layer with return_sequences=False for the final prediction
model.add(LSTM(64, activation='relu', return_sequences=False))

# Add a dense output layer with 9 units (assuming you want to predict 9 values)
model.add(Dense(2*3))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 5, 64)             18176     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 51590 (201.52 KB)
Trainable params: 51590 (201.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
batch_size = 32
epochs = 50

# Train the model
history = model.fit(X_training, y_training, batch_size=batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/50
30/30 [==============================] - 1s 10ms/step - loss: 0.2348 - accuracy: 0.1964 - val_loss: 0.0481 - val_accuracy: 0.3445
Epoch 2/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0294 - accuracy: 0.2763 - val_loss: 0.0233 - val_accuracy: 0.1933
Epoch 3/50
30/30 [==============================] - 0s 3ms/step - loss: 0.0147 - accuracy: 0.3382 - val_loss: 0.0194 - val_accuracy: 0.4412
Epoch 4/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0119 - accuracy: 0.4380 - val_loss: 0.0172 - val_accuracy: 0.6681
Epoch 5/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0105 - accuracy: 0.6670 - val_loss: 0.0160 - val_accuracy: 0.6681
Epoch 6/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0098 - accuracy: 0.6817 - val_loss: 0.0156 - val_accuracy: 0.6639
Epoch 7/50
30/30 [==============================] - 0s 4ms/step - loss: 0.0095 - accuracy: 0.6922 - val_loss: 0.0156 - val_accuracy: 0.6639
Epoch 8/50
30/30 [=

In [18]:
cap = cv2.VideoCapture(CAMERA_INDEX)
frame_buffer = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Converti il frame in scala di grigi
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Esegui il rilevamento delle pose con MediaPipe
    results = holistic.process(frame_rgb)

    # Estrai le coordinate della spalla destra, del gomito destro e della mano destra
    if results.pose_landmarks:
        right_shoulder = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
        right_wrist = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_WRIST]

        # Le coordinate sono normalizzate, quindi convertile in coordinate pixel
        height, width, _ = frame.shape
        shoulder_x, shoulder_y = int(right_shoulder.x * width), int(right_shoulder.y * height)
        elbow_x, elbow_y = int(right_elbow.x * width), int(right_elbow.y * height)
        wrist_x, wrist_y = int(right_wrist.x * width), int(right_wrist.y * height)

        # Disegna i punti sul frame
        cv2.circle(frame, (shoulder_x, shoulder_y), 5, (0, 0, 255), -1)
        cv2.circle(frame, (elbow_x, elbow_y), 5, (0, 255, 0), -1)
        cv2.circle(frame, (wrist_x, wrist_y), 5, (255, 0, 0), -1)

        # Estrai le coordinate x, y, e z e normalizzale
        shoulder_coords = [
            right_shoulder.x, 
            right_shoulder.y, 
            #right_shoulder.z
        ]
        elbow_coords = [
            right_elbow.x, 
            right_elbow.y, 
            #right_elbow.z
        ]
        wrist_coords = [
            right_wrist.x,
            right_wrist.y,
            #right_wrist.z
        ]

        # Logica per la predizione
        frame_buffer.append(
            shoulder_coords
            + elbow_coords
            + wrist_coords
        )

        if len(frame_buffer) >= window_len/2:
            prediction_input = np.array(frame_buffer)[np.newaxis, :, :]
            prediction = model.predict(prediction_input)
            print(prediction)

            # Draw magenta circles for the predicted coordinates
            pred_shoulder_x, pred_shoulder_y = int(prediction[0][0] * width), int(prediction[0][1] * height)
            pred_elbow_x, pred_elbow_y = int(prediction[0][2] * width), int(prediction[0][3] * height)
            pred_wrist_x, pred_wrist_y = int(prediction[0][4] * width), int(prediction[0][5] * height)

            cv2.circle(frame, (pred_shoulder_x, pred_shoulder_y), 5, (255, 0, 255), -1)
            cv2.circle(frame, (pred_elbow_x, pred_elbow_y), 5, (255, 0, 255), -1)
            cv2.circle(frame, (pred_wrist_x, pred_wrist_y), 5, (255, 0, 255), -1)

            frame_buffer = frame_buffer[1:]

    # Visualizza il frame con le coordinate
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 12ms/step
[[0.5977951  0.49150416 0.49105623 0.6124412  0.35901582 0.59284216]]
1/1 [==============================] - 0s 13ms/step
[[0.5955932  0.4978135  0.48648655 0.62792695 0.35115108 0.6135695 ]]
1/1 [==============================] - 0s 11ms/step
[[0.6016423  0.5127177  0.4907237  0.65718466 0.34911752 0.65024376]]
1/1 [==============================] - 0s 10ms/step
[[0.61159605 0.53455603 0.49966395 0.69101954 0.3514879  0.6875787 ]]
1/1 [==============================] - 0s 9ms/step
[[0.62237924 0.56701416 0.5048952  0.7372797  0.3523991  0.73374516]]
1/1 [==============================] - 0s 9ms/step
[[0.6369228  0.5980506  0.5183802  0.77235025 0.3622018  0.76345134]]
1/1 [==============================] - 0s 9ms/step
[[0.6410983  0.6219198  0.5312588  0.77583325 0.37735435 0.75200343]]
1/1 [==============================] - 0s 9ms/step
[[0.6461759  0.64216983 0.54110533 0.7765949  0.388473   0.7368471 ]]
1/1 [=======================